In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
%%bash

echo 'current working dir'
pwd

echo 'extracting training data .......'
unzip ../input/facial-keypoints-detection/training.zip

In [ ]:
%%bash

echo 'current working dir'
pwd

echo 'extracting test data .......'
unzip ../input/facial-keypoints-detection/test.zip

In [ ]:
! echo 'Working directory',$(pwd)
! ls

In [ ]:
import os 
work_dir = '/kaggle/working'
train_dir = os.path.join(work_dir,'training.csv')
test_dir = os.path.join(work_dir,'test.csv')

In [ ]:
train = pd.read_csv(train_dir)
test = pd.read_csv(test_dir)
sample_submission_csv = pd.read_csv('/kaggle/input/facial-keypoints-detection/SampleSubmission.csv')
lookup = pd.read_csv('/kaggle/input/facial-keypoints-detection/IdLookupTable.csv')

In [ ]:
print("Length of training data", len(train))
print("Length of testing data", len(test))

## Check for null values

In [ ]:
print(f'Feature \t\t\t Missing \t Percentage missing\n')
for k,v in train.isna().sum().items():
    print(f'{k !s:30} :{v :8} \t {round(v/len(train),2)}%')

Lets split the data into two parts <br>
Good_data = data that has no missing fields <br>
Bad_data = that has missing feilds -- we'll later inpute the missing  <br>

Lets start by working with good data and create a baseline model <br>


In [ ]:
good_data = train.dropna()
bad_data = train.drop(index=good_data.index)

In [ ]:
print('Shape of data we are going to work with',good_data.shape)

## Dataset information

In [ ]:
good_data.info()

The input image is given in the last field of the data files, and consists of a list of pixels (ordered by row), as integers in (0,255). The images are 96x96 pixels.

In [ ]:
# IMAGES
good_data['Image'].head()

## Images and targets
Lets create an util that accept a csv file and give us back the images and targets

In [ ]:
def get_images(data):
    images = []
    # prepare the image
    for img in iter(data.loc[:,'Image']):
        img = np.array(img.split(), dtype=float)
        img = img. reshape(96,96,1)
        images.append(img)
    return np.array(images)

def get_X_y(data):
    images = get_images(data)
    targets = np.array(data.iloc[:,:-1], dtype=float)
    return images, targets

In [ ]:
images, targets = get_X_y(good_data)

In [ ]:
print('Shape of images',images.shape)
print('Shape of targets',targets.shape)

In [ ]:
# lets create an util that will display the images
def display_images(img, feat):
    plt.imshow(img, cmap=plt.cm.gray);
    plt.scatter(feat[0::2], feat[1::2], c='r', marker='x')
    

In [ ]:
# lets test visualization
display_images(images[0],targets[0])

# Augmentation

We can add different augmenations like - 
* flipping 
* rotation
* cropping
* adding noise
* bluring
* brightness 

etc..

In [ ]:
# lets create an util to display the augumentation
def display_augmentation(img, feat, img_f, feat_f):
    plt.figure(figsize=(8,8))
    plt.subplot(2,2,1)
    plt.scatter(feat[0::2],-feat[1::2],c='r',marker='x')
    plt.subplot(2,2,2)
    plt.scatter(feat_f[0::2],-feat_f[1::2],c='r',marker='x')
    plt.subplot(2,2,3)
    display_images(img, feat)
    plt.subplot(2,2,4)
    display_images(img_f, feat_f)

In [ ]:
# sample image to test augmentation
image, feature = images[0].copy(), targets[0].copy()

### Flipping 

In [ ]:
flipped_img = np.fliplr(image)

In [ ]:
flipped_feat = feature.copy()
for i, v in enumerate(feature):
    if i % 2 ==0:
        v = 96-v
    flipped_feat[i]=v

In [ ]:
display_augmentation(image, feature, flipped_img, flipped_feat)

In [ ]:
# lets make an util that give us flipped images and targets
def flipping_augmentation(images, features):
    flipped_images = np.flip(images, axis=2)
    
    flipped_features = features.copy()
    for i, feat in enumerate(flipped_features):
        for j, val in enumerate(feat):
            if j%2==0:
                flipped_features[i][j] = 96-val
            
    return flipped_images, flipped_features

# let create an object to keep track of the augmentations
augmentation_functions = {
    'flip' : flipping_augmentation
}

In [ ]:
# lets see one more example
flip_imgs, flip_feats = flipping_augmentation(images[:5], targets[:5])

In [ ]:
display_augmentation(images[1], targets[1], flip_imgs[1],flip_feats[1])

### Cropping

In [ ]:
cropped_image = image.copy()

cropped_image[:,:10] = 0
cropped_image[:,86:] = 0
cropped_image[:10,:] = 0
cropped_image[86:,:] = 0


cropped_image.shape

In [ ]:
plt.imshow(cropped_image, cmap=plt.cm.gray);
plt.scatter(feature[0::2],feature[1::2],marker='x',c='r');

In [ ]:
# lets make an util that give us cropped images and targets
def crop_augmentation(images, targets):
    cropped_images = images.copy()

    for i in range(len(images)):
        cropped_images[i,:,:10] = 0
        cropped_images[i,:,86:] = 0
        cropped_images[i,:10,:] = 0
        cropped_images[i,86:,:] = 0

    return cropped_images, targets


augmentation_functions['crop']=crop_augmentation
    

### Rotation

In [ ]:
from scipy import ndimage, misc

img_45 = ndimage.rotate(image, 45, reshape=False)
plt.imshow(img_45 , cmap=plt.cm.gray);

In [ ]:
def rotate_points(points, angle):
    # shift points in the plane so that the center of rotation is at the origin
    points = points-48

    # rotation matrix
    theta = np.radians(angle)
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, s), (-s, c)))

    # rotate the points
    for i in range(0,len(points),2):
        xy = np.array([points[i],points[i+1]])
        xy_rot = R@xy
        points[i],points[i+1]= xy_rot

    #  shift again so the origin goes back to the desired center of rotation
    points = points+48
    return points

In [ ]:
# rotated feature points 
feat_45 = rotate_points(feature, 45)

In [ ]:
# lets see the rotation

display_augmentation(image, feature, img_45, feat_45)

In [ ]:
# lets make an util that give us rotated images and targets

def rotate_augmentation(images, features, angle):
    rotated_images = []
    for img in images:
        img_rot = ndimage.rotate(img, angle, reshape=False)
        rotated_images.append(img_rot)
        
    rotated_features=[]
    for feat in features:
        feat_rot = rotate_points(feat, angle)
        rotated_features.append(feat_rot)
        
    
    return np.array(rotated_images), np.array(rotated_features)


augmentation_functions['rotate'] = rotate_augmentation

In [ ]:
# check once more using a different angle
img_rot, feat_rot = rotate_augmentation(images[:5], targets[:5,:], angle=-45)

In [ ]:
display_augmentation(images[4], targets[4], img_rot[4], feat_rot[4])

In [ ]:
for i,theta in enumerate([10,20,30,40,-10,-20,-30,-40]):
    img, feat = rotate_augmentation(images[:10], targets[:10], theta)
    plt.subplot(2,4,i+1)
    display_images(img[i],feat[i])

### Brightness

In [ ]:
img = image.copy()
img = np.clip(img*2.5,0,255)
plt.imshow(img, cmap='gray');

In [ ]:
# lets create an util that adds brightness
def brightness_augmentation(images, features, factor=1.5):
    bright = []
    for img in images:
        bright.append(np.clip(img*factor, 0, 255))
    return np.array(bright), features

augmentation_functions['brightness'] = brightness_augmentation

In [ ]:
# lets see one more example
img, feat = brightness_augmentation(images[:5], targets[:5], factor=2.0)

display_augmentation(images[2], targets[2], img[2], feat[2])

### Adding noise 

In [ ]:
img = image.copy()
noise = np.random.randint(low=0, high=255, size=img.shape)
factor = 0.25
plt.imshow(img+(noise*factor), cmap='gray');

In [ ]:
# lets create an utility that adds noise to the image
def noise_augmentation(images, features, factor):
    augmented = []
    noise = np.random.randint(low=0, high=255, size=images.shape[1:])
    for img in images:
        img = img + (noise*factor)
        augmented.append(img)
    
    return np.array(augmented), features

augmentation_functions['noise'] = noise_augmentation

In [ ]:
# lets see one another example
img, feat = noise_augmentation(images[:5], targets[:5], factor=0.15)

display_augmentation(images[2], targets[2], img[2], feat[2])

## Lets prepare our training data 

In [ ]:
# images, targets = get_X_y(good_data)

print('Shape of image data',images.shape)
print('Shape of target data', targets.shape)

In [ ]:
%%time
# ADDING AUGMENTATION

def augmentation(img, feat , method):
    aug_img, aug_feat = method
    img = np.concatenate([img,aug_img])
    feat = np.concatenate([feat,aug_feat])
    return img, feat


# flip
method = flipping_augmentation(images, targets)
augmented_images, augmented_targets = augmentation(images, targets, method)

# crop
method = crop_augmentation(images, targets)
augmented_images, augmented_targets = augmentation(augmented_images, augmented_targets, method)

# rotate
for theta in [10,15,-10,-15]:
    method = rotate_augmentation(images, targets, angle=theta)
    augmented_images, augmented_targets = augmentation(augmented_images, augmented_targets, method)


# brightness
method = brightness_augmentation(images, targets, factor=2.0)
augmented_images, augmented_targets = augmentation(augmented_images, augmented_targets, method)

# noise
method = noise_augmentation(images, targets, factor=0.2)
augmented_images, augmented_targets = augmentation(augmented_images, augmented_targets, method)

# just for visual 
for k in augmentation_functions.keys():
    print(k,'>'*50)

In [ ]:
print('Shape of data after augmentation')
print('Shape of image data',augmented_images.shape)
print('Shape of target data', augmented_targets.shape)

In [ ]:
# lets check our data one last time before we start building models

def visualize_data(images, targets):
    plt.figure(figsize=(12,12))
    for i in range(10):
        idx = np.random.randint(images.shape[0])
        plt.subplot(2,5,i+1)
        display_images(images[idx], targets[idx])
        plt.axis('off')
    plt.subplots_adjust(bottom=0.5)
    plt.show()

visualize_data(augmented_images, augmented_targets)

# Modeling 

MobileNetV2 is very similar to the original MobileNet, except that it uses inverted residual blocks with bottlenecking features. It has a drastically lower parameter count than the original MobileNet. MobileNets support any input size greater than 32 x 32, with larger image sizes offering better performance.


In [ ]:
pretrained_model = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(96,96,3),include_top=False,weights='imagenet')
# pretrained_model = tf.keras.applications.resnet_v2.ResNet50V2(input_shape=(96,96,3),include_top=False,weights='imagenet')
pretrained_model.trainable=False

Each Keras Application expects a specific kind of input preprocessing. For MobileNetV2, call tf.keras.applications.mobilenet_v2.preprocess_input on your inputs before passing them to the model. mobilenet_v2.preprocess_input will scale input pixels between -1 and 1.

In [ ]:
augmented_images = tf.keras.applications.mobilenet_v2.preprocess_input(augmented_images)
# augmented_images = tf.keras.applications.resnet_v2.preprocess_input(augmented_images)

In [ ]:
# lets check our data to make sure everything is fine

visualize_data(augmented_images,augmented_targets)

In [ ]:
# lets create a dataset for training and validation
ds = tf.data.Dataset.from_tensor_slices((augmented_images,augmented_targets))
ds = ds.shuffle(buffer_size=augmented_targets.shape[0])
ds = ds.batch(64)
ds = ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
train_ds = ds.skip(10).shuffle(100)
val_ds = ds.take(10)

In [ ]:
# create a preprocessing layer
class ImageTile(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__(trainable = False)
        
    def call(self, inputs):
        return tf.tile(inputs,tf.constant([1,1,1,3]))

In [ ]:
model = tf.keras.Sequential([
    
    tf.keras.Input(shape=(96,96,1)),
    
    ImageTile(),
    
    pretrained_model,
    
    tf.keras.layers.GlobalMaxPooling2D(),
    
    tf.keras.layers.Dense(512),  
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Dense(256),  
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Dense(128),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Activation('relu'),

    tf.keras.layers.Dense(30)
])



In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy', 'mae', 'mse'])

In [ ]:
import math
# decaying learing rate
def decay_lr(epoch):
  return 0.01*math.pow(0.77,epoch)

lr_schedule = tf.keras.callbacks.LearningRateScheduler(decay_lr)

lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(patience=5)
    
    
early_stopping = tf.keras.callbacks.EarlyStopping(
                        monitor='val_loss',
                        patience=5,
                        restore_best_weights=True)

In [ ]:
history = model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[early_stopping,lr_on_plateau])

## Prediction

Lets use this model to make prediction

In [ ]:
test.head()

In [ ]:
test_images = get_images(test)

In [ ]:
print('Shape of test images', test_images.shape)

Predictions 

In [ ]:
test_preds = model.predict(test_images)

In [ ]:
print('Shape of test predictions', test_preds.shape)

In [ ]:
display_images(test_images[0],test_preds[0])

In [ ]:
visualize_data(test_images,test_preds)

In [ ]:
visualize_data(test_images,test_preds)        

## Submission

In [ ]:
feature_names = list(lookup['FeatureName'])
image_ids = list(lookup['ImageId']-1)
row_ids = list(lookup['RowId'])

feature_list = []
for feature in feature_names:
    feature_list.append(feature_names.index(feature))
    
predictions = []
for x,y in zip(image_ids, feature_list):
    predictions.append(test_preds[x][y])
    
row_ids = pd.Series(row_ids, name = 'RowId')
locations = pd.Series(predictions, name = 'Location')
locations = locations.clip(0.0,96.0)
submission_result = pd.concat([row_ids,locations],axis = 1)
submission_result.to_csv('submission.csv',index = False)